# xxx

In [3]:
from tqdm import tqdm
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


def set_style():
    sns.set_style('white')
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.rcParams['figure.figsize'] = [10, 10]

    pd.options.display.max_columns = 300
    pd.options.display.max_rows = 1000
    # plt.rcParams['figure.dpi'] = 200
    sns.set_palette('muted')  # 调色板颜色温和
    sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1})

set_style()
import sys

In [75]:
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import keras
from keras import layers
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from keras import Sequential

2.4.0


In [5]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Conv1D,MaxPooling1D, Flatten, BatchNormalization
from keras.layers import concatenate

In [6]:
!ls data

track1_round1_testA_20210222.csv  track1_round1_train_20210222.csv


In [7]:
test_data = open('data/track1_round1_testA_20210222.csv').readlines()
test_data = [data.strip().split('|,|') for data in test_data]
test_dataset = pd.DataFrame(test_data, columns=['report_id', 'description'])
test_dataset['words'] = test_dataset.description.apply(lambda x:x.split())

test_dataset.head()

,report_id,description,words
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[852, 328, 697, 538, 142, 355, 582, 800, 728, ..."
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[380, 358, 343, 654, 171, 832, 47, 832, 690, 6..."
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[751, 335, 834, 582, 717, 583, 585, 693, 623, ..."
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[623, 328, 649, 582, 488, 12, 578, 623, 538, 3..."
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[83, 293, 398, 797, 382, 363, 145, 424, 693, 6..."


In [8]:
train_data = open('data/track1_round1_train_20210222.csv','r').readlines()

train_data = [data.strip().split('|,|')  for data in train_data]

train_dataset = pd.DataFrame(train_data,columns=['report_id','description','label'])
train_dataset['words'] = train_dataset.description.apply(lambda x:x.split())

train_dataset.head()

,report_id,description,label,words
0,0,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[623, 328, 538, 382, 399, 400, 478, 842, 698, ..."
1,1,48 328 538 382 809 623 434 355 382 382 363 145...,,"[48, 328, 538, 382, 809, 623, 434, 355, 382, 3..."
2,2,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[623, 656, 293, 851, 636, 842, 698, 493, 338, ..."
3,3,48 328 380 259 439 107 380 265 172 470 290 693...,,"[48, 328, 380, 259, 439, 107, 380, 265, 172, 4..."
4,4,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[623, 328, 399, 698, 493, 338, 266, 14, 177, 4..."


In [9]:
dataset = train_dataset.append(test_dataset)
dataset.head()

,report_id,description,label,words
0,0,623 328 538 382 399 400 478 842 698 137 492 26...,2,"[623, 328, 538, 382, 399, 400, 478, 842, 698, ..."
1,1,48 328 538 382 809 623 434 355 382 382 363 145...,,"[48, 328, 538, 382, 809, 623, 434, 355, 382, 3..."
2,2,623 656 293 851 636 842 698 493 338 266 369 69...,15,"[623, 656, 293, 851, 636, 842, 698, 493, 338, ..."
3,3,48 328 380 259 439 107 380 265 172 470 290 693...,,"[48, 328, 380, 259, 439, 107, 380, 265, 172, 4..."
4,4,623 328 399 698 493 338 266 14 177 415 511 647...,16,"[623, 328, 399, 698, 493, 338, 266, 14, 177, 4..."


In [14]:
mlb = MultiLabelBinarizer()

df_label = pd.DataFrame(mlb.fit_transform(train_dataset.label.apply(lambda x:x.split())))
df_label.columns = mlb.classes_


label = df_label[[str(i) for i in range(17)]].values
label.shape

(10000, 17)

In [16]:
df_label[[str(i) for i in range(17)]].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [17]:
tokenizer=Tokenizer()  #创建一个Tokenizer对象
#fit_on_texts函数可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小
tokenizer.fit_on_texts(dataset['words'])
vocab=tokenizer.word_index #得到每个词的编号
# 将每个样本中的每个词转换为数字列表，使用每个词的编号进行编号

In [103]:
maxlen=100

train_data =tokenizer.texts_to_sequences(train_dataset['words'])
#将超过固定值的部分截掉，不足的在最前面用0填充
train_data_padded_seqs = pad_sequences(train_data, maxlen=maxlen, value = len(vocab))

test_data =tokenizer.texts_to_sequences(test_dataset['words'])
#将超过固定值的部分截掉，不足的在最前面用0填充
test_data_padded_seqs = pad_sequences(test_data, maxlen=maxlen, value = len(vocab))

In [104]:
train_data_padded_seqs.shape

(10000, 100)

## CNN

In [71]:
import keras

def my_score(y_true,y_pred):
    return 1 - keras.losses.binary_crossentropy(y_true, y_pred)

In [72]:
model = Sequential()
model.add(Embedding(len(vocab) + 1, 300, input_length=100)) #使用Embeeding层将每个词编码转换为词向量
model.add(Conv1D(256, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(128, 5, padding='same'))
model.add(MaxPooling1D(3, 3, padding='same'))
model.add(Conv1D(64, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(BatchNormalization())  # (批)规范化层
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(17, activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=5, batch_size=128, validation_split=0.1)
# y_predict = model.predict_classes(x_test_padded_seqs)  # 预测的是类别，结果就是类别号
# y_predict = list(map(str, y_predict))
# print('准确率', metrics.accuracy_score(y_test, y_predict))
# print('平均f1-score:', metrics.f1_score(y_test, y_predict, average='weighted'))


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 100, 300)          257700    
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 100, 256)          384256    
_________________________________________________________________
max_pooling1d_35 (MaxPooling (None, 34, 256)           0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 34, 128)           163968    
_________________________________________________________________
max_pooling1d_36 (MaxPooling (None, 12, 128)           0         
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 12, 64)            24640     
_________________________________________________________________
flatten_14 (Flatten)         (None, 768)             

## textCnn

In [105]:
main_input = Input(shape=(maxlen,), dtype='float64')
# 词嵌入（使用预训练的词向量）
embedder = Embedding(len(vocab) + 1, 512, input_length=maxlen, trainable=False)
embed = embedder(main_input)
# 词窗大小分别为3,4,5
cnn_ = Conv1D(256, 1, padding='same', strides=1, activation='relu')(embed)
cnn_ = MaxPooling1D(pool_size=maxlen)(cnn_)

cnn0 = Conv1D(256, 2, padding='same', strides=1, activation='relu')(embed)
cnn0 = MaxPooling1D(pool_size=maxlen-1)(cnn0)
cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
cnn1 = MaxPooling1D(pool_size=maxlen-2)(cnn1)
cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
cnn2 = MaxPooling1D(pool_size=maxlen-3)(cnn2)
cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
cnn3 = MaxPooling1D(pool_size=maxlen-4)(cnn3)

cnn4 = Conv1D(256, 6, padding='same', strides=1, activation='relu')(embed)
cnn4 = MaxPooling1D(pool_size=maxlen-5)(cnn4)
# 合并三个模型的输出向量
cnn = concatenate([cnn_, cnn0, cnn1, cnn2, cnn3, cnn4], axis=-1)
flat = Flatten()(cnn)
drop = Dropout(0.2)(flat)
main_output = Dense(17, activation='softmax')(drop)
model = Model(inputs=main_input, outputs=main_output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=20, batch_size=800,validation_split=0.1)


Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_44 (Embedding)        (None, 100, 512)     439808      input_23[0][0]                   
__________________________________________________________________________________________________
conv1d_86 (Conv1D)              (None, 100, 256)     131328      embedding_44[0][0]               
__________________________________________________________________________________________________
conv1d_87 (Conv1D)              (None, 100, 256)     262400      embedding_44[0][0]               
___________________________________________________________________________________________

In [108]:
model.fit(train_data_padded_seqs, label, epochs=20, batch_size=800,validation_split=0.1)

Epoch 1/20
12/12 [==============================] - 2s 187ms/step - loss: 0.0082 - my_score: 0.9918 - val_loss: 0.0314 - val_my_score: 0.9686
Epoch 2/20
12/12 [==============================] - 2s 182ms/step - loss: 0.0078 - my_score: 0.9922 - val_loss: 0.0311 - val_my_score: 0.9689
Epoch 3/20
12/12 [==============================] - 2s 183ms/step - loss: 0.0077 - my_score: 0.9923 - val_loss: 0.0306 - val_my_score: 0.9694
Epoch 4/20
12/12 [==============================] - 2s 182ms/step - loss: 0.0073 - my_score: 0.9927 - val_loss: 0.0305 - val_my_score: 0.9695
Epoch 5/20
12/12 [==============================] - 2s 182ms/step - loss: 0.0072 - my_score: 0.9928 - val_loss: 0.0304 - val_my_score: 0.9696
Epoch 6/20
12/12 [==============================] - 2s 182ms/step - loss: 0.0069 - my_score: 0.9931 - val_loss: 0.0300 - val_my_score: 0.9700
Epoch 7/20
12/12 [==============================] - 2s 205ms/step - loss: 0.0068 - my_score: 0.9932 - val_loss: 0.0316 - val_my_score: 0.9684
Epoch 

## Bilstm

In [76]:
inputs = Input(shape=(100,), dtype='float64')
embedder = Embedding(len(vocab) + 1, 300, input_length=100, trainable=False)
embed = embedder(inputs)
# Embed each integer in a 128-dimensional vector
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embed)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = Dense(17, activation='softmax')(x)
model = Model(inputs, outputs)
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
model.fit(train_data_padded_seqs, label, epochs=10, batch_size=200,validation_split=0.1)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_17 (Embedding)     (None, 100, 300)          257700    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_28 (Dense)             (None, 17)                2193      
Total params: 545,589
Trainable params: 287,889
Non-trainable params: 257,700
_________________________________________________________________
Epoch 1/10
45/45 [==============================] - 5s 60ms/step - loss: 0.4762 - my_score: 0.5238 - val_loss: 0.

## Transformer

In [87]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 256  # Hidden layer size in feed forward network inside transformer
maxlen=100
vocab_size = len(vocab) + 1

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(17, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[my_score])
# model.fit(train_data_padded_seqs, label, epochs=20, batch_size=200,validation_split=0.1)
model.fit(train_data_padded_seqs, label, epochs=20, batch_size=200)

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
token_and_position_embedding (None, 100, 256)          245504    
_________________________________________________________________
transformer_block_9 (Transfo (None, 100, 256)          658688    
_________________________________________________________________
global_average_pooling1d_9 ( (None, 256)               0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_68 (Dense)             (None, 100)               25700     
_________________________________________________________________
dropout_68 (Dropout)         (None, 100)               0  

## Prediction

In [109]:
test_pred = model.predict(test_data_padded_seqs)

In [110]:
result = ['|%s'%' '.join(['%.8f'%i for i in pred]) for pred in test_pred]

In [111]:
result[0]

'|0.00000358 0.00000552 0.00000059 0.00000006 0.00000000 0.00000000 0.00000000 0.96056551 0.00033139 0.03908781 0.00000473 0.00000014 0.00000023 0.00000026 0.00000001 0.00000024 0.00000004'

In [112]:
test_dataset['report_ID'] = test_dataset['report_id'].apply(lambda x:'%s|'%x)
test_dataset['Prediction'] =result
test_dataset.head()

,report_id,description,words,report_ID,Prediction
0,0,852 328 697 538 142 355 582 800 728 4 647 169 ...,"[852, 328, 697, 538, 142, 355, 582, 800, 728, ...",0|,|0.00000358 0.00000552 0.00000059 0.00000006 0...
1,1,380 358 343 654 171 832 47 832 690 693 48 563 ...,"[380, 358, 343, 654, 171, 832, 47, 832, 690, 6...",1|,|0.02543110 0.02346166 0.02847567 0.06428396 0...
2,2,751 335 834 582 717 583 585 693 623 328 107 38...,"[751, 335, 834, 582, 717, 583, 585, 693, 623, ...",2|,|0.00000646 0.00002015 0.00000131 0.00002256 0...
3,3,623 328 649 582 488 12 578 623 538 382 382 265...,"[623, 328, 649, 582, 488, 12, 578, 623, 538, 3...",3|,|0.01057643 0.00877395 0.01170924 0.01446421 0...
4,4,83 293 398 797 382 363 145 424 693 698 800 691...,"[83, 293, 398, 797, 382, 363, 145, 424, 693, 6...",4|,|0.00000294 0.00009296 0.00000019 0.00032965 0...


In [113]:
test_dataset[['report_ID','Prediction']].to_csv('submit.csv',index = False,header=None)